# Matrix free solvers

## Elliptic equation
We here study the general two-dimensional elliptic equation
\begin{align}
-\nabla\cdot( \chi \nabla_\perp \phi) = \rho
\end{align}
which in a two-dimensional Cartesian grid reads
\begin{align}
    -\frac{\partial}{\partial x} \left( \chi(x,y) \frac{\partial}{\partial x} \phi(x,y)\right) - \frac{\partial}{\partial y} \left( \chi(x,y) \frac{\partial}{\partial y} \phi(x,y)\right) = \rho(x,y)
    \label{eq:elliptic2d}
\end{align}
The task is to find a solution for $\phi$ for given $\rho$ and $\chi$.

## Discretization
We use discontinuous Galerkin (dG) methods to discretize
\begin{align}
\partial_x \rightarrow D_x
\end{align}

where $D_x$ is a block-sparse matrix.
We then have
\begin{align}
M \phi &= \rho \label{eq:matrix} \\
M &= D_x^T \chi D_x + D_y^T \chi D_y + J
\end{align}
We here see that $M$ is **self-adjoint**, which means that we can use a **conjugate gradient (CG)** solver.
## Problem

```cpp
// Pseudo-code:
Grid g;
Matrix dx = create_dx (g, bcx), dy = create_dy(g, bcy), j = create_jump();
DiagMatrix diag_chi = create_from_given_vector(chi);
// assembly of M requires four matrix-matrix multiplications and 2 additions
Matrix M = dx.transpose()*diag_chi*dx + dy.transpose()*diag_chi*dy + j;
// Now solve with CG
CG cg;
Vector phi = discretize_phi(g), rho = discretize_rho(g);
cg.solve( M, phi, rho, eps = 1e-8);
```
- In order to assemble $M$ four **matrix-matrix multiplcations** need to be performed
- This **takes longer than the entire CG solve**, at least in our initial tests (admittedly 10 years ago)


## Solution: matrix - free solvers
```{admonition}
A matrix-free solver is any solver for $M x = b$ that does **not require access** to the elements of the matrix $M_{ij}$
```
Matrix-free solvers are thus a subclass of available solvers

Examples of solvers that are **matrix-free**:
- All Krylov-subspace solvers are matrix-free. E.g. conjugate gradient (CG), LGMRES, BICG, etc.
- Fixed point iterations
- Chebyshev iteration


Examples of solvers that are **not matrix-free**
- Direct solvers; need to access $M_{ij}$ directly
- Jacobi iteration; because it needs to decompose $M = D + L + U$
- Gauss-Seidel iteration; needs to decompose $M = L_* + U$

## Example: Main loop of CG algorithm
To solve 
\begin{align}
Mx = b
\end{align}
the main loop of CG reads
\begin{align}
\alpha_k =& \frac{r_k^T \cdot r_k}{p_k^T \cdot ( \color{red}{M \cdot p_k})} \\
x_{k+1} = & x_k + \alpha_k p_k \\
r_{k+1} = & r_k - \alpha_k \color{red}{M \cdot p_k} \\
\beta_k = & \frac{r_{k+1}^T \cdot r_{k+1}}{r_k^T \cdot r_k} \\
p_{k+1} = & r_{k+1} + \beta_k p_k
\end{align}
To implement you only need to implement the **application of $M$ to a vector** rather than $M$ itself
```cpp
Grid g;
Matrix dx = create_dx (g, bcx), dy = create_dy(g, bcy), j = create_jump();
DiagMatrix diag_chi = create_from_given_vector(chi);

// Implement the effet of matrix without ever assembling it
Vector matrix_vector_product_with_elliptic_matrix( Vector phi)
{
    Vector dxP = dx*phi, dyP = dy*phi, JP = j*phi;
    Vector tempX = diag_chi*dxP, tempY = diag_chi*dyP;
    dxP = dx.transpose()*tempX, dy= dy.transpose()*tempY;
    return dxP + dyP + JP;
}

// In main CG loop
Vector Ap= matrix_vector_product_with_elliptic_matrix( p_k);
double alpha = r_old / p*Ap;
x = x + alpha*p;
r = r - alpha*Ap;
r_new = r*r;
double beta = r_new/r_old;
p = r + beta*p;
r_old = r_new;
```

## A manufactured example problem

We manufacture the solution 
\begin{align}
\chi &= 1 + A \sin (x)\sin(y) \\
\rho &= 2 \sin(x) \sin(y) (A\sin(x)\sin(y) +1)- A\sin^2(x)\cos^2(y) - A\cos^2(x)\sin^2(y) \\
\phi &= \sin(x)\sin(y)
\end{align}
for $A\in ]-1,1[$ and solve on the domain $[0,\pi]\times [0,2\pi]$ for Dirichlet boundary conditions in $x$ and periodic in $y$.
The **initital guess is zero**.

In [2]:
import simplesimdb
import numpy as np
import pandas as pd

In [7]:
def create_solver() :
    return {
        "type" : "PCG", # PCG, LGMRES, BICGSTABl, Multigrid
        "eps" : 1e-6,
        "preconditioner" :{
            "type" : "none" # None or diagonal
        },
        "max_iter" : 10000, # maximum number of iterations
        # for CG
        "check-every" : 10, # errror check every .. iterations
        # for LGMRES
        "inner_m" : 30,
        "outer_k" : 3,
        # for BICGSTABL
        "l_input" : 3
    }
def create_inputfile ():
    return {
        "grid" : {
            "n" : 3,
            "Nx" : 32, 
            "Ny" : 64,
        },
        "solver" : create_solver(),
        "equations":
        {
            "amp" : 0.999, # The closer to 1 the more difficult   
        },
        "elliptic" :
        {
            "jfactor" : 1,
            "direction" : "forward" # forward or centered

        }    
    }

In [8]:
def create_multigrid_input():
    return {
        "grid" : {
            "n" : 3,
            "Nx" : 32, 
            "Ny" : 64,
        },
        "solver" : 
        {
            "type" : "Multigrid-FAS", # Full approximation scheme
            "num_stages" : 3,
            "solvers" :
            [
                create_solver(),
                create_solver(),
                create_solver()
            ]      
        },
        "equations":
        {
            "amp" : 0.999, # The closer to 1 the more difficult   
        },
        "elliptic" :
        {
            "jfactor" : 1,
            "direction" : "forward" # forward or centered

        }    
    }


In [9]:
create_multigrid_input()

{'grid': {'n': 3, 'Nx': 32, 'Ny': 64},
 'solver': {'type': 'Multigrid-FAS',
  'num_stages': 3,
  'solvers': [{'type': 'PCG',
    'eps': 1e-06,
    'preconditioner': {'type': 'none'},
    'max_iter': 10000,
    'check-every': 10,
    'inner_m': 30,
    'outer_k': 3,
    'l_input': 3},
   {'type': 'PCG',
    'eps': 1e-06,
    'preconditioner': {'type': 'none'},
    'max_iter': 10000,
    'check-every': 10,
    'inner_m': 30,
    'outer_k': 3,
    'l_input': 3},
   {'type': 'PCG',
    'eps': 1e-06,
    'preconditioner': {'type': 'none'},
    'max_iter': 10000,
    'check-every': 10,
    'inner_m': 30,
    'outer_k': 3,
    'l_input': 3}]},
 'equations': {'amp': 0.999},
 'elliptic': {'jfactor': 1, 'direction': 'forward'}}